In [1]:
import json
import os
import glob
import pathlib
from skimage import img_as_uint, io
import tifffile as tf
from tqdm import tqdm
import re


In [85]:

# Specify the path to your JSON file
tif_folder = r"D:\UserData\PPM_calibration\2024.02.13\ppm_20x_colorImage_stitchingTest_1 - Copy"


data_folder = pathlib.Path(tif_folder) #pathlib.Path(tif_folder).parent
tif_strip_folder = data_folder.joinpath("strip_metadata")
if not os.path.exists(tif_strip_folder):
    os.mkdir(tif_strip_folder)

In [22]:
first_file = sorted( glob.glob(os.path.join(tif_folder,'*.tif')), key = lambda x: os.path.getmtime(x))[0]
first_file

'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos000_000.ome.tif'

In [23]:
mmfile = tf.TiffFile(first_file)

In [28]:
metadata =  mmfile.micromanager_metadata

In [80]:
pixelsize = float(re.findall('PhysicalSizeX="(\d*.\d*)"',mmfile.ome_metadata)[0])

In [31]:
data = metadata

In [35]:
data.keys()

dict_keys(['Summary', 'IndexMap', 'DisplaySettings', 'MajorVersion', 'Comments'])

In [40]:
len(data["Summary"]["StagePositions"])

9

In [55]:
def get_position_from_device(position):
    #print(position)
    for device in position[
                    "DevicePositions"
                ]:
        if (device['Device']=='XYStage'):
            x_,y_ = device['Position_um']
            return([x_,y_])
            

In [50]:
position['Label']

'1-Pos000_000'

In [52]:
position['GridCol']

0

In [56]:



pos = data["Summary"]["StagePositions"]
label_list = {}
position_list = {}
for ix, position in enumerate(pos):
    #print(ix, position)
    position_list.update(
        {
            (position["GridCol"], position["GridRow"]): get_position_from_device(position)
        }
    )
    label_list.update(
        {
            (position["GridCol"], position["GridRow"]): position[
                "Label"
            ]
        }
    )

position_list

{(0, 0): [25143.07744, -8182.639],
 (1, 0): [25723.0, -8182.639],
 (2, 0): [26302.92256, -8182.639],
 (2, 1): [26302.92256, -7708.0],
 (1, 1): [25723.0, -7708.0],
 (0, 1): [25143.07744, -7708.0],
 (0, 2): [25143.07744, -7233.361],
 (1, 2): [25723.0, -7233.361],
 (2, 2): [26302.92256, -7233.361]}

In [82]:
pixel_size = pixelsize

In [ ]:


save_config = True
save_tif = True

fl = glob.glob(tif_folder + r"\*.tif")

if save_config:
    # position_list = np.array(position_list)
    #pixel_size = 1.105
    # pixel_size = 1
    # with open(os.path.join(r"D:\__Data\20230110_Mike_tissue_4x", 'TileConfiguration.txt'), 'w') as text_file:
    with open(
        os.path.join(tif_strip_folder, "TileConfiguration.txt"),
        "w",
    ) as text_file:
        print("dim = 2", file=text_file)
        for pos in range(len(fl)):
            fn = pathlib.Path(fl[pos]).name
            # print(fn)
            xy = re.findall(r"Pos(\d{3})_(\d{3})", fn)
            xy = [(int(x), int(y)) for x, y in xy][0]
            # print(xy)
            # print(position_list[xy])
            x, y = position_list[xy]
            # x = int(position_list[pos, 0] / pixel_size)
            # y = int(position_list[pos, 1] / pixel_size)
            # print(pos,fl[pos])
            # print(x,y,label_list[xy])
            x = x / pixel_size
            y = y / pixel_size

            # break
            # print(f"{fn}; ; ({int(x)}, {int(y)})")
            # print("====")
            print(f"{fn}; ; ({int(x)}, {int(y)})", file=text_file)
            # if pos==2:
            #    break
    text_file.close()

In [87]:
fn

'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos000_000.ome.tif'

In [89]:
fl

['D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos000_000.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos000_001.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos000_002.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos001_000.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos001_001.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTest_1 - Copy\\ppm_20x_colorImage_stitchingTest_1_MMStack_1-Pos001_002.ome.tif',
 'D:\\UserData\\PPM_calibration\\2024.02.13\\ppm_20x_colorImage_stitchingTes

In [93]:

stitch_folder = tif_strip_folder

if save_tif:
    for fn in fl:
        img = tf.imread(fn)#.asarray()
        print(img.shape)
        nfn = os.path.join(stitch_folder, f"{pathlib.Path(fn).name}")
        io.imsave(nfn, img_as_uint(img), check_contrast=False)


OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: not an ome-tiff master file


(2200, 2688, 3)
(2200, 2688, 3)


OME series: not an ome-tiff master file
OME series: not an ome-tiff master file


(2200, 2688, 3)
(2200, 2688, 3)


OME series: not an ome-tiff master file
OME series: not an ome-tiff master file


(2200, 2688, 3)
(2200, 2688, 3)


OME series: not an ome-tiff master file
OME series: not an ome-tiff master file


(2200, 2688, 3)
(2200, 2688, 3)


OME series: not an ome-tiff master file


(2200, 2688, 3)


In [86]:

stitch_folder = tif_strip_folder

if save_tif:
    fn = fl[0]
    img = tf.TiffFile(fn)
    # positions_on_mmfile_stack = []
    labels_on_mmfile_stack = []
    for position in img.micromanager_metadata["Summary"]["StagePositions"]:
        # positions_on_mmfile_stack.append(get_xy_from_MM_position(position))
        labels_on_mmfile_stack.append(position["Label"])
        # print(position['Label'], get_xy_from_MM_position(position))
    img = img.asarray()
    print(img.shape)
    # print(positions_on_mmfile_stack)
    flip_y = True
    flip_x = False
    for ix in tqdm(range(img.shape[0]), total=img.shape[0]):
        img1 = img[ix, :, :, :]
        if flip_y:
            img1 = img1[::-1, :, :]
        if flip_x:
            img1 = img1[:, ::-1, :]

        label = labels_on_mmfile_stack[ix][
            2:
        ]  # remove the image-sequence tag on label eg : '2-Pos001_000'
        fnx = fn.replace("Pos000_000.ome.tif", f"{label}.ome.tif")
        nfn = os.path.join(stitch_folder, f"{pathlib.Path(fnx).name}")
        # print(pathlib.Path(fn).name, pathlib.Path(nfn).name)
        io.imsave(nfn, img_as_uint(img1), check_contrast=False)


read_json: invalid JSON
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)
OME series: cannot handle discontiguous storage (2200, 2688, 3) != (2200, 2688)


(2200, 2688, 3)


  0%|          | 0/2200 [00:00<?, ?it/s]


IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed